In [1]:

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pickle
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras import metrics
from keras.layers import Conv1D, GlobalMaxPooling1D, Activation


import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Using TensorFlow backend.


In [20]:
DATA_FILE = '/content/drive/My Drive/Colab Notebooks/sentiment-analysis-on-movie-reviews/train.tsv/train.tsv'
train = pd.read_csv(DATA_FILE,sep='\t', header = 0)
tags = train.Sentiment
texts = train.Phrase

num_max = 1000
# preprocess
le = LabelEncoder()
tags = le.fit_transform(tags)
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)
mat_texts = tok.texts_to_matrix(texts,mode='count')
print(tags[:5])
print(mat_texts[:5])
print(tags.shape,mat_texts.shape)


[1 2 2 2 2]
[[0. 3. 2. ... 0. 0. 0.]
 [0. 2. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(156060,) (156060, 1000)


In [22]:
from keras.preprocessing import sequence

def check_model(model,x,y):
    model.fit(x,y,batch_size=32,epochs=10,verbose=1,validation_split=0.2)


max_len = 100
cnn_texts_seq = tok.texts_to_sequences(texts)
print(cnn_texts_seq[0])
cnn_texts_mat = sequence.pad_sequences(cnn_texts_seq,maxlen=max_len)
print(cnn_texts_mat[0])
print(cnn_texts_mat.shape)


[2, 323, 3, 1, 9, 52, 8, 46, 13, 1, 8, 177, 46, 13, 1, 65, 3, 78, 668, 19, 576, 3, 78, 5, 57, 3, 2, 42]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   2 323   3   1   9  52   8  46  13   1   8 177  46  13   1  65   3  78
 668  19 576   3  78   5  57   3   2  42]
(156060, 100)


In [23]:
def get_cnn_model_v3():    # added filter
    model = Sequential()
    model.add(Embedding(1000,
                        20,
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(256, 3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

m = get_cnn_model_v3()
check_model(m,cnn_texts_mat,tags)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 124848 samples, validate on 31212 samples
Epoch 1/10
124848/124848 [==============================] - 69s 550us/step - loss: -1195240002.1739 - accuracy: 0.1710 - val_loss: -4481036936.3332 - val_accuracy: 0.1896
Epoch 2/10
124848/124848 [==============================] - 69s 549us/step - loss: -20996224152.4255 - accuracy: 0.1710 - val_loss: -39913035539.7832 - val_accuracy: 0.1896
Epoch 3/10
124848/124848 [==============================] - 69s 549us/step - loss: -97075845597.4205 - accuracy: 0.1710 - val_loss: -143224619512.1917 - val_accuracy: 0.1896
Epoch 4/10
124848/124848 [==============================] - 68s 548us/step - loss: -277065938678.1248 - accuracy: 0.1710 - val_loss: -360514133701.0442 - val_accuracy: 0.1896
Epoch 5/10
124848/124848 [==============================] - 70s 561us/step - loss: -620287648834.7969 - accuracy: 0.1710 - val_loss: -747640991310.9358 - val_accuracy: 0.1896
Epoch 6/10
124848/124848 [==============================] - 69s 551us/step - loss